<a href="https://colab.research.google.com/github/DarioneNazionale/KickLearning/blob/main/working_on_data/merging_historical_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Merging the historical stada in the dataset

### Imports

In [1]:
historical_data_path = "./drive/MyDrive/Project/Data"
data_path = "./drive/MyDrive/Project/Data/datasets"
destination_path = "./drive/MyDrive/Project/Data/complete"

In [2]:
import os
import numpy as np
from os.path import join
import pandas as pd
import time

### Fetching the historical data

In [3]:
historical_df = pd.read_csv(join(historical_data_path, "complete_historical_info.csv"))

### historical Info Function
Function to be then applied on all the dataset

In [4]:
def get_history(entry, historical_df):
    reduced_historical_df = historical_df[(historical_df["creator_id"] == entry["creator_id"])&(historical_df["id"] != entry["id"])&(historical_df["time"] < entry["launched_at"])]
    reduced_historical_df = reduced_historical_df.loc[reduced_historical_df.groupby('id')['time'].idxmax()]
    reduced_historical_df.drop(columns=["id", "creator_id"], inplace=True)
    
    if len(reduced_historical_df) == 0: return pd.Series(dtype=int)
    else: return pd.Series([reduced_historical_df[col].tolist() for col in reduced_historical_df.columns], index=reduced_historical_df.columns)

## Applying on all the dataset
Merging the historical data with the actual one file by file

In [5]:
file_list = sorted(filter(lambda file: file[:4]=="file", os.listdir(data_path)), reverse=True)

start = time.perf_counter()
for i, file_name in enumerate(file_list, start=1):

    df = pd.read_csv(os.path.join(data_path, file_name))

    df_history = df[["id", "creator_id", "year", "launched_at"]].apply(lambda entry: get_history(entry, historical_df), axis=1)

    df = pd.concat((df, df_history), axis=1)

    df.to_csv(join(destination_path, file_name), index=False)

    print(f"Time elapsed: {time.perf_counter()-start}; remaning time: {(time.perf_counter()-start)/i*(len(file_list)-i)}")
    print(f"Done {round(i/len(file_list)*100)}% untill now, in total {i} files", end="\n\n")

    # debbug print
    if i == 1: print(df.head(10))

print(f"Time elapsed in total: {time.perf_counter()-start}")

Time elapsed: 493.0190586969984; remaning time: 7888.305214511987
Done 6% untill now, in total 1 files

     deadline  ...                             usd_pledged
0  1410373095  ...                                     NaN
1  1365980340  ...                                     NaN
2  1521039624  ...                                     NaN
3  1520689842  ...  [146592.0, 281241.0, 77754.0, 60869.0]
4  1513702800  ...                               [94415.0]
5  1525367592  ...                    [1128.3752550000004]
6  1512662340  ...                        [687263.0729832]
7  1522353600  ...                              [44653.25]
8  1511982005  ...                                     NaN
9  1414784642  ...                                     NaN

[10 rows x 21 columns]
Time elapsed: 1171.2432008629985; remaning time: 8784.324177772505
Done 12% untill now, in total 2 files

Time elapsed: 1852.3279223110003; remaning time: 8644.197134444665
Done 18% untill now, in total 3 files

Time elapse